# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajatsharma369007/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rajatsharma369007/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Create a database engine
engine = create_engine('sqlite:///disaster_response_pipeline.db')

# Load data from the database into a DataFrame
query = 'SELECT * FROM full_dataset'
df = pd.read_sql(query, engine)

# Display the loaded data
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [3]:
df.shape

(26216, 40)

In [4]:
rows_with_twos = df.isin([2]).any(axis=1)
filtered_df = df[~rows_with_twos]
filtered_df.shape

(26027, 40)

In [5]:
filtered_df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [6]:
# X represents features  
# y represents labels

engine = create_engine('sqlite:///disaster_response_pipeline.db')
X = filtered_df['message']
y = filtered_df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    """
    Tokenizes message data
    
    INPUT:
       text (string): message text data
    
    OUTPUT:
        (DataFrame) clean_messages: array of tokenized message data
    """
        
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)     # Remove non-alphanumeric characters using regex
    words = word_tokenize(text)                    # Tokenize the text into words
    words = [word.lower() for word in words]       # Convert words to lowercase
    stop_words = set(stopwords.words('english'))   # Remove stopwords
    words = [word for word in words if word not in stop_words]
    stemmer = PorterStemmer()                      # Initialize Porter Stemmer
    words = [stemmer.stem(word) for word in words] # Apply stemming
    
    return words


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42)
pipe.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8d01af9f70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [10]:
y_test

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
21047,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12462,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
25670,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13913,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0
14720,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9564,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2895,1,1,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20602,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
X_train.shape, X_test.shape

((18218,), (7809,))

In [12]:
y_train.shape , y_test.shape

((18218, 36), (7809, 36))

In [13]:
from sklearn.metrics import classification_report
y_test_pred = pipe.predict(X_test)      
y_test_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [14]:
from sklearn.metrics import classification_report

for i, label in enumerate(y.columns):
    true_label = y_test.iloc[:, i]
    pred_label = y_test_pred[:, i]
    
    report = classification_report(true_label, pred_label)
    print(f"Label: {label}")
    print(report)

Label: related
              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1821
           1       0.84      0.95      0.89      5988

    accuracy                           0.82      7809
   macro avg       0.77      0.67      0.70      7809
weighted avg       0.81      0.82      0.80      7809

Label: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6453
           1       0.84      0.48      0.61      1356

    accuracy                           0.89      7809
   macro avg       0.87      0.73      0.77      7809
weighted avg       0.89      0.89      0.88      7809

Label: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      7766
           1       0.00      0.00      0.00        43

    accuracy                           0.99      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      0.

In [15]:
print(classification_report(y_test, y_test_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5988
               request       0.84      0.48      0.61      1356
                 offer       0.00      0.00      0.00        43
           aid_related       0.76      0.70      0.73      3225
          medical_help       0.61      0.05      0.10       599
      medical_products       0.78      0.06      0.11       369
     search_and_rescue       0.75      0.03      0.05       216
              security       0.00      0.00      0.00       148
              military       0.65      0.04      0.08       249
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.37      0.52       474
                  food       0.86      0.55      0.67       877
               shelter       0.84      0.28      0.42       684
              clothing       0.82      0.19      0.30       123
                 money       1.00      

In [253]:
# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Feature', 'Weighted Precision', 'Weighted Recall', 'Weighted F1-Score'])

for i, label in enumerate(y.columns):
    true_label = y_test.iloc[:, i]
    pred_label = y_test_pred[:, i]
    report_dict = classification_report(true_label, pred_label, output_dict=True)
    
    # Extract the weighted precision, recall, and f1-score
    weighted_precision = report_dict['weighted avg']['precision']
    weighted_recall = report_dict['weighted avg']['recall']
    weighted_f1_score = report_dict['weighted avg']['f1-score']
    
    # Append the results to the DataFrame
    results_df = results_df.append({'Feature': label, 'Weighted Precision': weighted_precision, 'Weighted Recall': weighted_recall, 'Weighted F1-Score': weighted_f1_score}, ignore_index=True)

results_df


,Feature,Weighted Precision,Weighted Recall,Weighted F1-Score
0,related,0.810689,0.823153,0.803160
1,request,0.886053,0.891151,0.878398
2,offer,0.989017,0.994494,0.991748
3,aid_related,0.782878,0.784095,0.781994
4,medical_help,0.913820,0.927135,0.899040
5,medical_products,0.947937,0.954412,0.934593
6,search_and_rescue,0.968501,0.973364,0.961842
7,security,0.972051,0.981048,0.971913
8,military,0.961207,0.968882,0.955218
9,child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.